In [1]:
import pyodbc
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
name_db = 'IndexDataBase'

data_conection = ("Driver={SQL Server};"
                 "Server=DESKTOP-0EKP870;"
                 f"Database={name_db};")
cnxn = pyodbc.connect(data_conection)

------------
**Importação dos nomes das tabelas**

In [3]:
informationsTables = f'''
SELECT * FROM INFORMATION_SCHEMA.TABLES
'''
informations_tables_df = pd.read_sql(informationsTables, cnxn)

In [4]:
display(informations_tables_df.head(3))

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,IndexDataBase,dbo,Table3373,BASE TABLE
1,IndexDataBase,dbo,Table6460,BASE TABLE
2,IndexDataBase,dbo,Table1135,BASE TABLE


---------
**Ordenando os dados referentes aos nomes**

In [5]:
names = informations_tables_df['TABLE_NAME'].tolist()

In [6]:
for i, name in enumerate(names):
    names[i] = int(name[5:])

In [7]:
names_orderned = sorted(names)
for i, name in enumerate(names_orderned):
    names_orderned[i] = 'Table'+str(name)

In [8]:
def values_from_matrix (matrix):
    valeus_outside_diagonal = []
    for i, dataset in enumerate(matrix):
        for j in range(i):
            valeus_outside_diagonal.append(dataset[j])
    return valeus_outside_diagonal

In [9]:
def estimate_covariance (stocks_frame):
    list_prices = []
    for column in stocks_frame.columns:
        list_prices.append(stocks_frame.iloc[:][column].tolist())
    matrix_prices = np.array(list_prices)
    
    cov_matrix = np.cov(matrix_prices,bias=True)
    corr_matrix = np.corrcoef(matrix_prices)
    
    mean_cov = np.mean(values_from_matrix (cov_matrix))
    mean_corr = np.mean(values_from_matrix (corr_matrix))
    return mean_cov, mean_corr

In [10]:
all_cov_mean = []
all_corr_mean = []
for name_table in tqdm(names_orderned):
    ImportTable = f'''
    SELECT * FROM {name_table}
    '''
    stocks_frame = pd.read_sql(ImportTable, cnxn)
    mean_cov, mean_corr = estimate_covariance (stocks_frame)
    all_cov_mean.append(mean_cov)
    all_corr_mean.append(mean_corr)

100%|██████████████████████████████████████████████████████████████████████████████| 8150/8150 [02:44<00:00, 49.50it/s]


In [11]:
df_covariance = pd.DataFrame(list(zip(all_cov_mean, all_corr_mean)), columns=['Covariance',                                                                            'Correlation'])
df_covariance.head(3)

,Covariance,Correlation
0,1791.567553,0.288998
1,1788.359821,0.289429
2,1789.219188,0.290246


In [12]:
df_covariance.to_excel(r'data/CovarianceCorrelation.xlsx')